In [29]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, confusion_matrix , mean_absolute_error , classification_report
from sklearn.preprocessing import StandardScaler
from google.colab import drive

import matplotlib.pyplot as plt

In [2]:
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Food_delivery_prediction/Food_Delivery_Time_Prediction.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Order_ID,Customer_Location,Restaurant_Location,Distance,Weather_Conditions,Traffic_Conditions,Delivery_Person_Experience,Order_Priority,Order_Time,Vehicle_Type,Restaurant_Rating,Customer_Rating,Delivery_Time,Order_Cost,Tip_Amount
0,ORD0001,"(17.030479, 79.743077)","(12.358515, 85.100083)",1.57,Rainy,Medium,4,Medium,Afternoon,Car,4.1,3.0,26.22,1321.10,81.54
1,ORD0002,"(15.398319, 86.639122)","(14.174874, 77.025606)",21.32,Cloudy,Medium,8,Low,Night,Car,4.5,4.2,62.61,152.21,29.02
2,ORD0003,"(15.687342, 83.888808)","(19.594748, 82.048482)",6.95,Snowy,Medium,9,High,Night,Bike,3.3,3.4,48.43,1644.38,64.17
3,ORD0004,"(20.415599, 78.046984)","(16.915906, 78.278698)",13.79,Cloudy,Low,2,Medium,Evening,Bike,3.2,3.7,111.63,541.25,79.23
4,ORD0005,"(14.786904, 78.706532)","(15.206038, 86.203182)",6.72,Rainy,High,6,Low,Night,Bike,3.5,2.8,32.38,619.81,2.34


In [3]:
print(f"{df.shape[0]} and {df.shape[1]}")
df.info()

200 and 15
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Order_ID                    200 non-null    object 
 1   Customer_Location           200 non-null    object 
 2   Restaurant_Location         200 non-null    object 
 3   Distance                    200 non-null    float64
 4   Weather_Conditions          200 non-null    object 
 5   Traffic_Conditions          200 non-null    object 
 6   Delivery_Person_Experience  200 non-null    int64  
 7   Order_Priority              200 non-null    object 
 8   Order_Time                  200 non-null    object 
 9   Vehicle_Type                200 non-null    object 
 10  Restaurant_Rating           200 non-null    float64
 11  Customer_Rating             200 non-null    float64
 12  Delivery_Time               200 non-null    float64
 13  Order_Cost              

In [4]:
print(df.isnull().sum())
df.describe()

Order_ID                      0
Customer_Location             0
Restaurant_Location           0
Distance                      0
Weather_Conditions            0
Traffic_Conditions            0
Delivery_Person_Experience    0
Order_Priority                0
Order_Time                    0
Vehicle_Type                  0
Restaurant_Rating             0
Customer_Rating               0
Delivery_Time                 0
Order_Cost                    0
Tip_Amount                    0
dtype: int64


,Distance,Delivery_Person_Experience,Restaurant_Rating,Customer_Rating,Delivery_Time,Order_Cost,Tip_Amount
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,11.498050,5.250000,3.738500,3.686500,70.494950,1046.488700,46.616650
std,6.841755,2.745027,0.703021,0.697063,29.830694,548.568922,29.361706
min,0.520000,1.000000,2.500000,2.600000,15.230000,122.300000,1.240000
25%,6.090000,3.000000,3.200000,3.100000,46.997500,553.270000,21.602500
50%,10.265000,5.000000,3.800000,3.700000,72.775000,1035.950000,47.530000
75%,16.497500,8.000000,4.300000,4.300000,96.650000,1543.125000,70.245000
max,24.900000,10.000000,5.000000,5.000000,119.670000,1997.420000,99.740000


In [5]:
df = df.drop(columns=['Order_ID'], errors='ignore')
print(df.columns)
df.head()

Index(['Customer_Location', 'Restaurant_Location', 'Distance',
       'Weather_Conditions', 'Traffic_Conditions',
       'Delivery_Person_Experience', 'Order_Priority', 'Order_Time',
       'Vehicle_Type', 'Restaurant_Rating', 'Customer_Rating', 'Delivery_Time',
       'Order_Cost', 'Tip_Amount'],
      dtype='object')


,Customer_Location,Restaurant_Location,Distance,Weather_Conditions,Traffic_Conditions,Delivery_Person_Experience,Order_Priority,Order_Time,Vehicle_Type,Restaurant_Rating,Customer_Rating,Delivery_Time,Order_Cost,Tip_Amount
0,"(17.030479, 79.743077)","(12.358515, 85.100083)",1.57,Rainy,Medium,4,Medium,Afternoon,Car,4.1,3.0,26.22,1321.10,81.54
1,"(15.398319, 86.639122)","(14.174874, 77.025606)",21.32,Cloudy,Medium,8,Low,Night,Car,4.5,4.2,62.61,152.21,29.02
2,"(15.687342, 83.888808)","(19.594748, 82.048482)",6.95,Snowy,Medium,9,High,Night,Bike,3.3,3.4,48.43,1644.38,64.17
3,"(20.415599, 78.046984)","(16.915906, 78.278698)",13.79,Cloudy,Low,2,Medium,Evening,Bike,3.2,3.7,111.63,541.25,79.23
4,"(14.786904, 78.706532)","(15.206038, 86.203182)",6.72,Rainy,High,6,Low,Night,Bike,3.5,2.8,32.38,619.81,2.34


In [6]:
print(df['Traffic_Conditions'].unique())
print(df['Order_Priority'].unique())

['Medium' 'Low' 'High']
['Medium' 'Low' 'High']


In [7]:
traffic_mapping = {
    'Low': 0,
    'Medium': 1,
    'High': 2
}

if df['Traffic_Conditions'].dtype == 'object':
    df['Traffic_Conditions'] = df['Traffic_Conditions'].map(traffic_mapping)

priority_mapping = {
    'Low': 0,
    'Medium': 1,
    'High': 2
}


if df['Order_Priority'].dtype == 'object':
    df['Order_Priority'] = df['Order_Priority'].map(priority_mapping)


print(df[['Traffic_Conditions', 'Order_Priority']].head())



   Traffic_Conditions  Order_Priority
0                   1               1
1                   1               0
2                   1               2
3                   0               1
4                   2               0


In [8]:
print(df['Weather_Conditions'].unique())
print(df['Vehicle_Type'].unique())
print(df['Order_Time'].unique())

['Rainy' 'Cloudy' 'Snowy' 'Sunny']
['Car' 'Bike' 'Bicycle']
['Afternoon' 'Night' 'Evening' 'Morning']


In [9]:
df = pd.get_dummies(
    df,
    columns=['Weather_Conditions', 'Vehicle_Type', 'Order_Time'],
    drop_first=True
)
df.head()

,Customer_Location,Restaurant_Location,Distance,Traffic_Conditions,Delivery_Person_Experience,Order_Priority,Restaurant_Rating,Customer_Rating,Delivery_Time,Order_Cost,Tip_Amount,Weather_Conditions_Rainy,Weather_Conditions_Snowy,Weather_Conditions_Sunny,Vehicle_Type_Bike,Vehicle_Type_Car,Order_Time_Evening,Order_Time_Morning,Order_Time_Night
0,"(17.030479, 79.743077)","(12.358515, 85.100083)",1.57,1,4,1,4.1,3.0,26.22,1321.10,81.54,True,False,False,False,True,False,False,False
1,"(15.398319, 86.639122)","(14.174874, 77.025606)",21.32,1,8,0,4.5,4.2,62.61,152.21,29.02,False,False,False,False,True,False,False,True
2,"(15.687342, 83.888808)","(19.594748, 82.048482)",6.95,1,9,2,3.3,3.4,48.43,1644.38,64.17,False,True,False,True,False,False,False,True
3,"(20.415599, 78.046984)","(16.915906, 78.278698)",13.79,0,2,1,3.2,3.7,111.63,541.25,79.23,False,False,False,True,False,True,False,False
4,"(14.786904, 78.706532)","(15.206038, 86.203182)",6.72,2,6,0,3.5,2.8,32.38,619.81,2.34,True,False,False,True,False,False,False,True


In [10]:
df.describe()

,Distance,Traffic_Conditions,Delivery_Person_Experience,Order_Priority,Restaurant_Rating,Customer_Rating,Delivery_Time,Order_Cost,Tip_Amount
count,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,11.498050,0.85000,5.250000,0.915000,3.738500,3.686500,70.494950,1046.488700,46.616650
std,6.841755,0.80044,2.745027,0.813069,0.703021,0.697063,29.830694,548.568922,29.361706
min,0.520000,0.00000,1.000000,0.000000,2.500000,2.600000,15.230000,122.300000,1.240000
25%,6.090000,0.00000,3.000000,0.000000,3.200000,3.100000,46.997500,553.270000,21.602500
50%,10.265000,1.00000,5.000000,1.000000,3.800000,3.700000,72.775000,1035.950000,47.530000
75%,16.497500,2.00000,8.000000,2.000000,4.300000,4.300000,96.650000,1543.125000,70.245000
max,24.900000,2.00000,10.000000,2.000000,5.000000,5.000000,119.670000,1997.420000,99.740000


In [11]:
numeric_df = df.select_dtypes(include=['number'])
corr_matrix = numeric_df.corr()
corr_matrix


,Distance,Traffic_Conditions,Delivery_Person_Experience,Order_Priority,Restaurant_Rating,Customer_Rating,Delivery_Time,Order_Cost,Tip_Amount
Distance,1.000000,0.134181,0.017562,-0.007464,0.064676,0.003430,-0.075143,-0.022275,-0.006466
Traffic_Conditions,0.134181,1.000000,-0.053745,-0.143230,-0.026299,-0.018958,-0.055840,-0.032744,0.152003
Delivery_Person_Experience,0.017562,-0.053745,1.000000,0.131150,-0.004492,-0.122447,-0.019098,0.061015,-0.069218
Order_Priority,-0.007464,-0.143230,0.131150,1.000000,0.105974,-0.098679,-0.004982,-0.062279,-0.111948
Restaurant_Rating,0.064676,-0.026299,-0.004492,0.105974,1.000000,0.101661,-0.091855,-0.106870,-0.035759
Customer_Rating,0.003430,-0.018958,-0.122447,-0.098679,0.101661,1.000000,-0.021952,0.142336,-0.074884
Delivery_Time,-0.075143,-0.055840,-0.019098,-0.004982,-0.091855,-0.021952,1.000000,-0.009307,-0.029154
Order_Cost,-0.022275,-0.032744,0.061015,-0.062279,-0.106870,0.142336,-0.009307,1.000000,0.034462
Tip_Amount,-0.006466,0.152003,-0.069218,-0.111948,-0.035759,-0.074884,-0.029154,0.034462,1.000000


In [12]:
df['Rush_Hour'] = ((df['Order_Time_Evening'] == 1) |
                   (df['Order_Time_Night'] == 1)).astype(int)
df[['Order_Time_Evening', 'Order_Time_Night', 'Rush_Hour']].head()

,Order_Time_Evening,Order_Time_Night,Rush_Hour
0,False,False,0
1,False,True,1
2,False,True,1
3,True,False,1
4,False,True,1


In [13]:
x = df.drop(columns=['Delivery_Time', 'Customer_Location', 'Restaurant_Location'])
y = df['Delivery_Time']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(160, 17)
(40, 17)
(160,)
(40,)


In [14]:
numeric_cols = [
    'Distance',
    'Delivery_Person_Experience',
    'Restaurant_Rating',
    'Customer_Rating',
    'Order_Cost',
    'Tip_Amount'
]
scaler = StandardScaler()
x_train[numeric_cols] = scaler.fit_transform(x_train[numeric_cols])
x_test[numeric_cols] = scaler.transform(x_test[numeric_cols])
x_train[numeric_cols].describe()

,Distance,Delivery_Person_Experience,Restaurant_Rating,Customer_Rating,Order_Cost,Tip_Amount
count,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02
mean,-1.082467e-16,8.604228e-17,-7.216450e-16,-4.496403e-16,7.771561e-17,5.551115e-18
std,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00
min,-1.578957e+00,-1.604241e+00,-1.794831e+00,-1.534937e+00,-1.659214e+00,-1.545965e+00
25%,-8.490643e-01,-8.740034e-01,-7.676409e-01,-8.648901e-01,-8.931215e-01,-8.775437e-01
50%,-1.861208e-01,2.213533e-01,1.128074e-01,1.675118e-02,-2.443081e-02,2.261598e-02
75%,7.286101e-01,9.515911e-01,8.465144e-01,8.631268e-01,9.018181e-01,8.102665e-01
max,1.937336e+00,1.681829e+00,1.873704e+00,1.850565e+00,1.720683e+00,1.848263e+00


In [15]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_test)

In [16]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 1018.3776259818181
Mean Absolute Error: 27.175573008773334
R-squared: -0.10107828422455722


In [17]:
df['Delivery_Time'].median()

72.775

In [19]:
df['Delivery_Status'] = (df['Delivery_Time'] > 72.775).astype(int)
df['Delivery_Status'].value_counts()

,count
Delivery_Status,
0,100
1,100


In [20]:
X = df.drop(columns=['Delivery_Time', 'Delivery_Status', 'Customer_Location', 'Restaurant_Location'])
y = df['Delivery_Status']

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(X_train.shape)
print(X_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(160, 17)
(40, 17)
Delivery_Status
0    80
1    80
Name: count, dtype: int64
Delivery_Status
0    20
1    20
Name: count, dtype: int64


In [24]:
from sklearn.preprocessing import StandardScaler

numeric_cols = [
    'Distance',
    'Delivery_Person_Experience',
    'Restaurant_Rating',
    'Customer_Rating',
    'Order_Cost',
    'Tip_Amount'
]

scaler = StandardScaler()

X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
X_train[numeric_cols].describe()

,Distance,Delivery_Person_Experience,Restaurant_Rating,Customer_Rating,Order_Cost,Tip_Amount
count,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02
mean,-4.440892e-17,-5.551115e-18,-3.330669e-17,-5.551115e-18,-6.106227e-17,-2.081668e-17
std,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00
min,-1.639375e+00,-1.528820e+00,-1.792591e+00,-1.508480e+00,-1.645461e+00,-1.499278e+00
25%,-7.328064e-01,-7.986373e-01,-8.021114e-01,-9.272478e-01,-9.103407e-01,-9.442131e-01
50%,-1.452296e-01,-6.845462e-02,4.687090e-02,-5.539874e-02,8.125282e-04,3.854871e-02
75%,7.135653e-01,1.026819e+00,8.958532e-01,8.164503e-01,8.954472e-01,8.227728e-01
max,2.023011e+00,1.757002e+00,1.744835e+00,1.978916e+00,1.733251e+00,1.815293e+00


In [28]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [30]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.475


In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.55      0.51        20
           1       0.47      0.40      0.43        20

    accuracy                           0.47        40
   macro avg       0.47      0.48      0.47        40
weighted avg       0.47      0.47      0.47        40



In [32]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[11  9]
 [12  8]]
